In [ ]:
%load_ext autoreload
%autoreload 2

In [117]:
import sys
sys.path.insert(0, '..//src/')

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
from matplotlib.colors import LinearSegmentedColormap
import matplotlib as mpl
import seaborn as sns
import sys
import random
import main
from common import enums, viz_functions
from pathlib import Path

random.seed(42)

# blues = [x for x in reversed(sns.color_palette("Blues_d", 11))]
# sns.set_palette(sns.color_palette("Paired", 10))
# sns.set_style("white")
sns.set_palette(sns.color_palette("colorblind", 15))
plt.rcParams['font.family'] = 'Times New Roman'
font_size = 12
params = {'axes.labelsize': font_size+2,
          'axes.titlesize':font_size+4,
          'legend.fontsize': font_size, 
          'xtick.labelsize': font_size,
          'ytick.labelsize': font_size,
          'font.size': font_size}
plt.rcParams.update(params)
sns.set_style("whitegrid")
export_fig_format = "pdf"
fig_format = "png"
dpi = 300
figsize = (12, 3.5)
figsize_map = (16, 8)
sns.palplot(sns.color_palette())

markers = ['P', 'o', '^', 's', '*', 'v']
linewidth = 0.1
edgecolor = 'black'

print(os.getcwd())
computer_path= ""
for x in os.getcwd().split(os.path.sep):
    print(x)
    computer_path=computer_path+x+os.path.sep
    if x == "sceac10":
        break    
    else: computer_path = os.getcwd().split(os.path.sep)[0]+os.path.sep
print(computer_path)


In [119]:
PATH_ONEDRIVE = Path(r'D:\Profile data Cardiff\OneDrive - Cardiff University')
path_gb_data = PATH_ONEDRIVE / r'04 - Projects\00 - Final data\Annual_demand'

In [ ]:
scenario = 'after'
gb_df = pd.read_csv(path_gb_data /
                    f"GB_Annual_heat_demand_{scenario}_EE_LSOA.csv", index_col=0)
gb_df.head()

In [ ]:
list_columns = [x for x in gb_df.columns if f"Average heat demand {scenario}" in x]
list_columns

In [122]:
## Remove outliers
for c in list_columns:
  max_value = gb_df[c].quantile(0.99)
  filt = gb_df[c] > max_value
  gb_df.loc[filt, c] = max_value

In [ ]:
to_plot_df = pd.DataFrame()
to_plot_df.index = gb_df['Lsoa11cd'].values
to_plot_df[f"Total heat demand {scenario} energy efficiency measures (MWh)"] = gb_df[
    list_columns].sum(axis=1).values/1000
to_plot_df.head()

In [ ]:
map_df = viz_functions.get_UK_LSOA_map()
map_df = pd.merge(map_df, to_plot_df, left_on='LSOA11CD', right_index=True)
map_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(figsize_map))
cmap_1 = 'magma_r'
target = f"Total heat demand {scenario} energy efficiency measures (MWh)"
map_obj = viz_functions.plot_map(map_df,
                                 target,
                                 ax,
                                 vmin=0,
                                 vmax=400,
                                 cmap=cmap_1,
                                 legend=True)

ax.set_title(f"Residential heat demand\n{scenario} energy efficiency measures (MWh)")
fig